In [5]:
# things to do/doubts/discussions
# how to save subclassing api
# do regularization
# hyperparameter tuning
# how to save the best model among all the epochs
# should we do augmentation of validation data ?
# should we shuffle repeat, prefetch etc ?
# All values are already between -1 to +1. should we do scaling ?
# if we are adding scaling of data, how to ensure that when it is being tested by brauer our code would give out rescaled data

In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn import preprocessing
# import tensorflow_datasets as tfds

from sklearn.metrics import mean_squared_error
from tensorflow import keras
from keras import layers
# from sklearn import preprocessing
# Make numpy values easier to read.
# np.set_printoptions(precision=3, suppress=True)

# physical_devices = tf.config.list_physical_devices("gpu")
print("# GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

tf.keras.backend.set_floatx('float64')
tf.compat.v1.enable_eager_execution()

# np.random.seed(101)
# tf.random.set_seed(101)

import warnings
warnings.filterwarnings('ignore')

# GPUs Available:  1


In [7]:
# loading the data of data

leakage_train_100 = pd.read_csv("leakage_dataset_train_100.csv")
leakage_train_1000 = pd.read_csv("leakage_dataset_train_1000.csv")
leakage_val_1000 = pd.read_csv("leakage_dataset_validation_1000.csv")

In [8]:
# data exploration
# print(leakage_train_100.columns)
# print(leakage_train_1000.columns)
# print(leakage_val_1000.columns)

# def data_description(datset):
#     print(datset['y1'].describe())
#     print(datset['y2'].describe())
#     print(datset['mfc1'].describe())
#     print(datset['mfc2'].describe())
#     print(datset['mfc3'].describe())
#     print(datset['mfc4'].describe())

# data_description(leakage_train_100)
# data_description(leakage_train_1000)
# data_description(leakage_val_1000)

# sns.set()
# cols = ['y1', 'y2', 'mfc1', 'mfc2', 'mfc3', 'mfc4']
# sns.pairplot(leakage_train_1000[cols], size = 2.5)
# plt.show()

# missing data
# def missingdata(df_train):
#     total = df_train.isnull().sum().sort_values(ascending=False)
#     percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
#     missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
#     print(missing_data.head(20))

# missingdata(leakage_val_1000)

# histogram and normal probability plot
# from scipy.stats import norm
# from scipy import stats

# sns.distplot(leakage_val_1000['y1'], fit=norm)
# fig = plt.figure()
# res = stats.probplot(leakage_val_1000['y1'], plot=plt)

In [9]:
def load_data(train_data):

    train_ds = train_data
    val_ds = leakage_val_1000

    train_ds = train_ds.sample(frac=1)
    val_ds = val_ds.sample(frac=1)

    X_train = train_ds.iloc[:,2:].to_numpy()
    Y_train = train_ds.iloc[:,:2]

    X_validation = val_ds.iloc[:,2:].to_numpy()
    Y_validation = val_ds.iloc[:,:2]

    Y_train = Y_train.to_numpy()
    Y_validation = Y_validation.to_numpy()
    return X_train, Y_train, X_validation, Y_validation

X_train, Y_train, X_validation, Y_validation = load_data(leakage_train_100)


#test dataset
X_test = np.array([[0.25, 0.25, 0.25,0.25], 
                    [0.35, 0.15, 0.25,0.25], 
                    [0.25, 0.25, 0.15,0.35],
                    [0.15, 0.35, 0.25,0.25],
                    [0.25, 0.25, 0.35,0.15],
                    [0.4, 0.1, 0.25,0.25],
                    [0.05, 0.45, 0.25,0.25],
                    [0.45, 0.95, 0.25,0.25],
                    [0.25, 0.25, 0.45,0.05],
                    [0.25, 0.25, 0.05,0.45]])
# scX = preprocessing.StandardScaler()
# stY = preprocessing.StandardScaler()

# def data_scaling(X_train, X_validation, Y_train, Y_validation, scX, stY):
#     X_train = scX.fit_transform(X_train)
#     X_validation = scX.transform(X_validation)

#     Y_train = stY.fit_transform(Y_train)
#     Y_validation = stY.transform(Y_validation)

#     return X_train, X_validation, Y_train, Y_validation

# X_train, X_validation, Y_train, Y_validation = data_scaling(X_train, X_validation, Y_train, Y_validation, scX, stY)

X_train = tf.convert_to_tensor(X_train)
X_validation = tf.convert_to_tensor(X_validation)
Y_train = tf.convert_to_tensor(Y_train)
Y_validation = tf.convert_to_tensor(Y_validation)
X_test = tf.convert_to_tensor(X_test)

num_rows, num_cols = X_train.shape

TypeError: list indices must be integers or slices, not tuple

In [ ]:
batch_size = 64
epochs = 1000
# steps_per_epoch = sum(train_occurences) / batch_size
starter_learning_rate = 1e-1
end_learning_rate = 1e-8
decay_steps = epochs * 3
scheduler = tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate= starter_learning_rate,
    decay_steps= decay_steps,
    end_learning_rate= end_learning_rate,
    power=1)
    
def learning_curves(history)   :
    sns.set_style('darkgrid', {'axes.facecolor': '.9'})
    sns.set_context('notebook')

    # your code
    ### Learning curves
    history_frame = pd.DataFrame(history.history)
    history_frame.plot(figsize=(8, 5))
    plt.show()

def prediction_accuracy(predictions, Y_validation): 
    predictions = predictions.transpose()
    Y_validation = tf.transpose(Y_validation)
    y1 = predictions[0]
    y2 = predictions[1]
    y1_validation = Y_validation[0]
    y2_validation = Y_validation[1]
    fig, axs = plt.subplots(2)
    # print(y1_validation.shape, y1.shape)
    # print(y2_validation.shape, y2.shape)
    # fig.suptitle('')
    axs[0].scatter(y1_validation, y1)
    axs[0].set_title('y1')
    axs[1].scatter(y2_validation, y2)
    axs[1].set_title('y2')
    for ax in axs.flat:
        ax.set(xlabel='true value', ylabel='predicted value')
    for ax in axs.flat:
        ax.label_outer()

    print("rmse of y1: ", mean_squared_error(y1_validation, y1, squared=False))
    print("rmse of y2: ", mean_squared_error(y2_validation, y2, squared=False))

# def dataloading(data):
#     data = data.repeat()
#     data = data.shuffle(buffer_size=1024, seed=0)
#     data = data.batch(batch_size=batch_size)
#     data = data.prefetch(buffer_size=1)
#     return data

In [ ]:
# data Augmentation
# Requires cleaning up

def rotation_matrix(angle):
    theta = np.radians(angle)
    c, s = np.cos(theta), np.sin(theta)
    R = np.array(((c, -s), (s, c)))
    return R

def Augmentation_clock(x,y):

    x = x.copy()
    y = y.copy()
    # print(y)
    y_aug = np.transpose(np.matmul(rotation_matrix(-90), np.transpose(y)))
    # print(y_aug)

    temp = x.copy()
    x0 = temp[:,0]
    x1 = temp[:,1]
    x2 = temp[:,2]
    x3 = temp[:,3]

    x[:,0] = x3
    x[:,1] = x0
    x[:,2] = x1
    x[:,3] = x2
 
    return x,y_aug

def Augmentation_flip(x,y):
    x = x.copy()
    y = y.copy()
    x = np.flip(x, axis=1)
    y[:,1] = -1 * y[:,1]
    return x,y

def Augmentation_anticlock(x,y):

    x = x.copy()
    y = y.copy()
    y_aug = np.transpose(np.matmul(rotation_matrix(90), np.transpose(y)))

    temp = x.copy()
    x0 = temp[:,0]
    x1 = temp[:,1]
    x2 = temp[:,2]
    x3 = temp[:,3]

    x[:,0] = x1
    x[:,1] = x2
    x[:,2] = x3
    x[:,3] = x0
 
    return x,y_aug

def data_augmentation(x,y):
    x = x.numpy()
    y = y.numpy()
    x_aug1,y_aug1 = Augmentation_clock(x, y)
    x_aug2,y_aug2 = Augmentation_clock(x_aug1,y_aug1)
    x_aug3,y_aug3 = Augmentation_clock(x_aug2,y_aug2)
    x_aug4,y_aug4 = Augmentation_flip(x_aug3,y_aug3)
    x_aug5,y_aug5 = Augmentation_clock(x_aug4,y_aug4)
    x_aug6,y_aug6 = Augmentation_clock(x_aug5,y_aug5)
    x_aug7,y_aug7 = Augmentation_clock(x_aug6,y_aug6)
    X_train_Aug = np.concatenate((x, x_aug1, x_aug2, x_aug3, x_aug4, x_aug5, x_aug6, x_aug7))
    Y_train_Aug = np.concatenate((y, y_aug1, y_aug2, y_aug3, y_aug4, y_aug5, y_aug6, y_aug7))

    X_train_Aug = tf.convert_to_tensor(X_train_Aug)
    Y_train_Aug = tf.convert_to_tensor(Y_train_Aug)

    return X_train_Aug, Y_train_Aug

X_train_Aug, Y_train_Aug = data_augmentation(X_train, Y_train)
# X_validation_Aug, Y_validation_Aug = data_augmentation(X_validation, Y_validation)


In [ ]:
# A neural network with subclassing api on Normal data

class Hidden_layer(layers.Layer):
    def __init__(self, units):
        super(Hidden_layer, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1],self.units), initializer=tf.keras.initializers.HeUniform, trainable=True)

    def call(self, inputs):
        x = tf.matmul(inputs, self.W)
        return x

class Output_layer(layers.Layer):
    def __init__(self, units):
        super(Output_layer, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1],self.units), initializer=tf.keras.initializers.HeUniform, trainable=True)

    def call(self, inputs):
        x = tf.matmul(inputs, self.W)
        return x

class MyReLU(layers.Layer):
    def __init__(self):
        super(MyReLU, self).__init__()

    def call(self, x):
        return tf.math.maximum(x, 0)

class MyModel(keras.Model):  # model.fit, model.evalute, model.predict
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = Hidden_layer(4)
        self.dense2_1 = Output_layer(2)
        self.relu = MyReLU()

    def call(self, x):
        x = self.relu(self.dense1(x))
        return self.dense2_1(x)

    def build_graph(self):
        x = tf.keras.layers.Input(shape=(1,4))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))

model = MyModel()
model.build_graph().summary()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=scheduler),
              loss='mean_squared_error',
              metrics = [tf.keras.metrics.MeanSquaredError()]
)

history = model.fit(X_train, Y_train, epochs=epochs, batch_size= batch_size, verbose=2, validation_data=(X_validation, Y_validation),
                    callbacks=[keras.callbacks.EarlyStopping(patience=50)],
                    # shuffle=True
                    )

predictions = model.predict(X_validation)
prediction_accuracy(predictions, Y_validation)
learning_curves(history)
evaluate_train = model.evaluate(X_train, Y_train, batch_size=batch_size)

evaluate_train = model.evaluate(X_train, Y_train, batch_size=batch_size)
evaluate_validation = model.evaluate(X_validation,Y_validation, batch_size=batch_size)

In [ ]:
# A neural network with subclassing api on Augmented data

class Hidden_layer(layers.Layer):
    def __init__(self, units):
        super(Hidden_layer, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1],self.units), initializer=tf.keras.initializers.HeUniform, trainable=True)

    def call(self, inputs):
        x = tf.matmul(inputs, self.W)
        return x

class Output_layer(layers.Layer):
    def __init__(self, units):
        super(Output_layer, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1],self.units), initializer=tf.keras.initializers.HeUniform, trainable=True)

    def call(self, inputs):
        x = tf.matmul(inputs, self.W)
        return x

class MyReLU(layers.Layer):
    def __init__(self):
        super(MyReLU, self).__init__()

    def call(self, x):
        return tf.math.maximum(x, 0)

class MyModel(keras.Model):  # model.fit, model.evalute, model.predict
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = Hidden_layer(4)
        self.dense2_1 = Output_layer(2)
        self.relu = MyReLU()

    def call(self, x):
        x = self.relu(self.dense1(x))
        return self.dense2_1(x)

    def build_graph(self):
        x = tf.keras.layers.Input(shape=(1,4))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))

model = MyModel()
model.build_graph().summary()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=scheduler),
              loss='mean_squared_error',
              metrics = [tf.keras.metrics.MeanSquaredError()]
)

history = model.fit(X_train_Aug, Y_train_Aug, epochs=epochs, batch_size= batch_size, verbose=2, validation_data=(X_validation, Y_validation),
                    callbacks=[keras.callbacks.EarlyStopping(patience=50)],
                    # shuffle=True
                    )

predictions = model.predict(X_validation)
prediction_accuracy(predictions, Y_validation)
learning_curves(history)

evaluate_train = model.evaluate(X_train_Aug, Y_train_Aug, batch_size=batch_size)
evaluate_validation = model.evaluate(X_validation,Y_validation, batch_size=batch_size)

In [ ]:
# Equivariant NN - on Normal data

class Hidden_layer(layers.Layer):
    def __init__(self, units):
        super(Hidden_layer, self).__init__()
        self.units = units
        self.W = None

    def build(self, input_shape):
        self.a = self.add_weight(shape=(1,), initializer=tf.keras.initializers.HeUniform, trainable=True)
        self.b = self.add_weight(shape=(1,), initializer=tf.keras.initializers.HeUniform, trainable=True)
        self.c = self.add_weight(shape=(1,), initializer=tf.keras.initializers.HeUniform, trainable=True)
        self.a_matrix = tf.constant([[1,0,0,0], [0,1,0,0], [0,0,1,0],[0,0,0,1]], dtype=tf.float64)
        self.b_matrix = tf.constant([[0,1,0,1], [1,0,1,0], [0,1,0,1],[1,0,1,0]], dtype=tf.float64)
        self.c_matrix = tf.constant([[0,0,1,0], [0,0,0,1], [1,0,0,0],[0,1,0,0]], dtype=tf.float64)
    
    def call(self, inputs):
        self.W = tf.multiply(self.a, self.a_matrix) + tf.multiply(self.b, self.b_matrix) + tf.multiply(self.c, self.c_matrix)
        x = tf.matmul(inputs, self.W)
        # tf.print(self.W)
        return x

class Output_layer(layers.Layer):
    def __init__(self, units):
        super(Output_layer, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.d = self.add_weight(shape=(1,), initializer=tf.keras.initializers.HeUniform, trainable=True)
        self.d_matrix = tf.constant([[1,-1], [-1,-1], [-1,1],[1,1]], dtype=tf.float64)
        # self.d_matrix = tf.constant([[1,-1, -1,1], [-1,-1,1,1]], dtype=tf.float64)

    def call(self, inputs):
        self.W = tf.multiply(self.d, self.d_matrix)
        x = tf.matmul(inputs, self.W)
        # tf.print(tf.transpose(self.W))
        return x

class MyReLU(layers.Layer):
    def __init__(self):
        super(MyReLU, self).__init__()

    def call(self, x):
        return tf.math.maximum(x, 0)

class MyModel(keras.Model):  # model.fit, model.evalute, model.predict
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = Hidden_layer(4)
        self.dense2_1 = Output_layer(2)
        # self.dense2_2 = Output_layer(1)
        self.relu = MyReLU()

    def call(self, x):
        x = self.relu(self.dense1(x))
        # return {"y1" :self.dense2_1(x),"y2" : self.dense2_2(x)}
        return self.dense2_1(x)

    def build_graph(self):
        x = tf.keras.layers.Input(shape=(1,4))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))

model = MyModel()
model.build_graph().summary()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=scheduler),
              loss='mean_squared_error',
              metrics = [tf.keras.metrics.MeanSquaredError()]
)

history = model.fit(X_train, Y_train, epochs=epochs, batch_size= batch_size, verbose=2, validation_data=(X_validation, Y_validation),
                    callbacks=[keras.callbacks.EarlyStopping(patience=50)],
                    # shuffle=True
                    )

predictions = model.predict(X_validation)
prediction_accuracy(predictions, Y_validation)
learning_curves(history)

evaluate_train = model.evaluate(X_train, Y_train, batch_size=batch_size)
evaluate_validation = model.evaluate(X_validation,Y_validation, batch_size=batch_size)

y_pred = model.predict(X_test)
print(y_pred)

In [ ]:
# Equivariant NN - on Normal data

class Hidden_layer(layers.Layer):
    def __init__(self, units):
        super(Hidden_layer, self).__init__()
        self.units = units
        self.W = None

    def build(self, input_shape):
        self.a = self.add_weight(shape=(1,), initializer=tf.keras.initializers.HeUniform, trainable=True)
        self.b = self.add_weight(shape=(1,), initializer=tf.keras.initializers.HeUniform, trainable=True)
        self.c = self.add_weight(shape=(1,), initializer=tf.keras.initializers.HeUniform, trainable=True)
        self.a_matrix = tf.constant([[1,0,0,0], [0,1,0,0], [0,0,1,0],[0,0,0,1]], dtype=tf.float64)
        self.b_matrix = tf.constant([[0,1,0,1], [1,0,1,0], [0,1,0,1],[1,0,1,0]], dtype=tf.float64)
        self.c_matrix = tf.constant([[0,0,1,0], [0,0,0,1], [1,0,0,0],[0,1,0,0]], dtype=tf.float64)
    
    def call(self, inputs):
        self.W = tf.multiply(self.a, self.a_matrix) + tf.multiply(self.b, self.b_matrix) + tf.multiply(self.c, self.c_matrix)
        x = tf.matmul(inputs, self.W)
        # tf.print(self.W)
        return x

class Output_layer(layers.Layer):
    def __init__(self, units):
        super(Output_layer, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.d = self.add_weight(shape=(1,), initializer=tf.keras.initializers.HeUniform, trainable=True)
        self.d_matrix = tf.constant([[1,-1], [-1,-1], [-1,1],[1,1]], dtype=tf.float64)
        # self.d_matrix = tf.constant([[1,-1, -1,1], [-1,-1,1,1]], dtype=tf.float64)

    def call(self, inputs):
        self.W = tf.multiply(self.d, self.d_matrix)
        x = tf.matmul(inputs, self.W)
        # tf.print(tf.transpose(self.W))
        return x

class MyReLU(layers.Layer):
    def __init__(self):
        super(MyReLU, self).__init__()

    def call(self, x):
        return tf.math.maximum(x, 0)

class MyModel(keras.Model):  # model.fit, model.evalute, model.predict
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = Hidden_layer(4)
        self.dense2_1 = Output_layer(2)
        # self.dense2_2 = Output_layer(1)
        self.relu = MyReLU()

    def call(self, x):
        x = self.relu(self.dense1(x))
        # return {"y1" :self.dense2_1(x),"y2" : self.dense2_2(x)}
        return self.dense2_1(x)

    def build_graph(self):
        x = tf.keras.layers.Input(shape=(1,4))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))

model = MyModel()
model.build_graph().summary()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=scheduler),
              loss='mean_squared_error',
              metrics = [tf.keras.metrics.MeanSquaredError()]
)

history = model.fit(X_train_Aug, Y_train_Aug, epochs=epochs, batch_size= batch_size, verbose=2, validation_data=(X_validation, Y_validation),
                    callbacks=[keras.callbacks.EarlyStopping(patience=50)],
                    # shuffle=True
                    )

predictions = model.predict(X_validation)
prediction_accuracy(predictions, Y_validation)
learning_curves(history)

evaluate_train = model.evaluate(X_train_Aug, Y_train_Aug, batch_size=batch_size)
evaluate_validation = model.evaluate(X_validation,Y_validation, batch_size=batch_size)

y_pred = model.predict(X_test)
print(y_pred)

In [ ]:
# # Equivariant NN - by averaging weights (doesnt work) and Kernel constraint
# # how to make constraint work on subclassing api
# # Equivariance NN with subclassing API - experiment with layer constraint
# # These function make use of the subclassing Api and custom layer functionality of tensorflow
# # experiment

# class Hidden_layer(layers.Layer):
#     def __init__(self, units, kernel_constraint):
#         super(Hidden_layer, self).__init__()
#         self.units = units
#         self.kernel_constraint = kernel_constraint

#     def build(self, input_shape):
#         self.W = self.add_weight(
#             shape=(input_shape[-1], self.units),
#             # initializer=tf.keras.initializers.GlorotNormal, 
#             initializer=tf.keras.initializers.HeUniform,
#             trainable=True, constraint=self.kernel_constraint) 
            
#     def call(self, inputs):
#         x = tf.matmul(inputs, self.W)
#         return x

# class Hidden_layer_constraint(tf.keras.constraints.Constraint):
#     # def __init__(self):
#     #     self.d = None

#     # @tf.function
#     def __call__(self, w):
#         a = tf.reduce_mean([w[0,0], w[1,1], w[2,2], w[3,3]])
#         b = tf.reduce_mean([w[0,1], w[1,0], w[1,2], w[2,1], w[0,3], w[3,0], w[2,3], w[3,2]])
#         c = tf.reduce_mean([w[0,2], w[1,3], w[2,0], w[3,1]])
        
#         row_indices = tf.constant([0, 1, 2, 3])
#         col_indices = tf.constant([0, 1, 2, 3])
#         w = tf.tensor_scatter_nd_update(w, tf.stack([row_indices, col_indices], axis=1), tf.repeat(a, tf.shape(row_indices)[0]))

#         row_indices = tf.constant([0, 0, 1, 1, 2, 2, 3, 3])
#         col_indices = tf.constant([1, 3, 2, 0, 1, 3, 0, 2])
#         w = tf.tensor_scatter_nd_update(w, tf.stack([row_indices, col_indices], axis=1), tf.repeat(b, tf.shape(row_indices)[0]))

#         row_indices = tf.constant([0, 1, 3, 2])
#         col_indices = tf.constant([2, 3, 1, 0])
#         w = tf.tensor_scatter_nd_update(w, tf.stack([row_indices, col_indices], axis=1), tf.repeat(c, tf.shape(row_indices)[0]))

#         return w

# class Output_layer_constraint(tf.keras.constraints.Constraint):
#     # def __init__(self):
#     #     self.d = None

#     # @tf.function
#     def __call__(self, w):
#         self.d = tf.reduce_mean(w)
#         w = w/w * self.d
#         row_indices = tf.constant([0, 1, 1, 2])
#         col_indices = tf.constant([1, 0, 1, 0])
#         w = tf.tensor_scatter_nd_update(w, tf.stack([row_indices, col_indices], axis=1), tf.repeat(-self.d, tf.shape(row_indices)[0]))
#         return w

# class Output_layer(layers.Layer):
#     def __init__(self, units, kernel_constraint):
#         super(Output_layer, self).__init__()
#         self.units = units
#         self.kernel_constraint = kernel_constraint

#     def build(self, input_shape):
#         self.W = self.add_weight(
#             shape=(input_shape[-1], self.units),
#             # initializer=tf.keras.initializers.GlorotNormal, 
#             initializer=tf.keras.initializers.HeUniform,
#             trainable=True, constraint=self.kernel_constraint
#         )

#     def call(self, inputs):
#         x = tf.matmul(inputs, self.W)
#         return x

# class MyReLU(layers.Layer):
#     def __init__(self):
#         super(MyReLU, self).__init__()

#     def call(self, x):
#         return tf.math.maximum(x, 0)

# class MyModel(keras.Model):  # model.fit, model.evalute, model.predict
#     def __init__(self):
#         super(MyModel, self). __init__()
#         self.dense1 = Hidden_layer(4, kernel_constraint=Hidden_layer_constraint())
#         self.dense2 = Hidden_layer(4, kernel_constraint=Hidden_layer_constraint())
#         # self.dense2_1 = tf.keras.layers.Dense(2, kernel_constraint=Output_layer_constraint())
#         self.dense3_1 = Output_layer(2, kernel_constraint=Output_layer_constraint())
#         self.relu = MyReLU()

#     def call(self, x):
#         x = self.relu(self.dense1(x))
#         x = self.relu(self.dense2(x))
#         # return {"y1" :self.dense2_1(x),"y2" : self.dense2_2(x)}
#         return self.dense3_1(x)

#     def build_graph(self):
#         x = tf.keras.layers.Input(shape=(1,4))
#         return tf.keras.Model(inputs=[x], outputs=self.call(x))

# model = MyModel()
# model.build_graph().summary()

# starter_learning_rate = 1e-0
# end_learning_rate = 1e-7
# decay_steps = 100
# scheduler = tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate= starter_learning_rate,
#     decay_steps= decay_steps,
#     end_learning_rate= end_learning_rate,
#     power=1)

# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
#               loss=tf.keras.losses.mean_squared_error,
#               metrics = [tf.keras.metrics.MeanSquaredError()]
# )

# history = model.fit(X_train, Y_train, epochs=epochs, batch_size= batch_size, verbose=2, validation_data=(X_validation, Y_validation),
#                     # callbacks=[keras.callbacks.EarlyStopping(patience=10)]
#                     )

# print("output layer weight matrix")
# print(model.layers[2].weights)

# print("Hidden layer weight matrix")
# print(model.layers[1].weights)

# print("Hidden layer weight matrix")
# print(model.layers[0].weights)

# predictions = model.predict(X_validation)
# prediction_accuracy(predictions, Y_validation)
# learning_curves(history)

In [ ]:
# # ----------------------------- Functional Tuning - Option 2: using Keras Tuner ------------------------------
# # Goal: tune the learning rate
# import keras_tuner as kt

# # 1. Define the general architecture of the model through a creation user-defined function
# def model_builder(hp):
#   model = Sequential()
#   model.add(Dense(10, activation='relu', input_shape=(n_features,)))
#   model.add(Dense(8, activation='relu'))
#   model.add(Dense(1))
#   hp_learning_rate = hp.Choice('learning_rate', values = [1e-1, 1e-2, 1e-3, 1e-4]) # Tuning the learning rate (four different values to test: 0.1, 0.01, 0.001, 0.0001)
#   #optimizer = RMSprop(learning_rate = hp_learning_rate) 
#   optimizer = tf.keras.optimizers.Adam(learning_rate = hp_learning_rate) # Defining the optimizer
#   model.compile(loss='mse',metrics=['mse'], optimizer=optimizer)                   # Compiling the model 
#   return model                                                                     # Returning the defined model

# # 2. Define the hyperparameters grid to be validated
# tuner_rs = kt.RandomSearch(
#               model_builder,                # Takes hyperparameters (hp) and returns a Model instance
#               objective = 'mse',            # Name of model metric to minimize or maximize
#               seed = 42,                    # Random seed for replication purposes
#               max_trials = 5,               # Total number of trials (model configurations) to test at most. Note that the oracle may interrupt the search before max_trial models have been tested.
#               directory='random_search')    # Path to the working directory (relative).

# # 3. Run the GridSearchCV process
# tuner_rs.search(X_train, y_train, epochs=10, validation_split=0.2, verbose=1)

In [ ]:
# # Not important Experiment 2 - learned how to make a neural network with subclassing api
# # These function make use of the subclassing Api and custom layer functionality of tensorflow
# # experiment

# class Hidden_layer(layers.Layer):
#     def __init__(self, units):
#         super(Hidden_layer, self).__init__()
#         self.units = units

#     def build(self, input_shape):
#         self.a = self.add_weight(shape=(1,), initializer=tf.keras.initializers.GlorotNormal, trainable=False)
#         self.b = self.add_weight(shape=(1,), initializer=tf.keras.initializers.GlorotNormal, trainable=False)
#         self.c = self.add_weight(shape=(1,), initializer=tf.keras.initializers.GlorotNormal, trainable=False)
#         x= tf.Variable(
#             [self.a, self.b, self.c, self.b, 
#              self.b, self.a, self.b, self.c, 
#              self.c, self.b, self.a, self.b, 
#              self.b, self.c, self.b, self.a])
#         x = tf.reshape(x, shape=(input_shape[-1],self.units))
#         self.W = tf.Variable(x, shape=(input_shape[-1],self.units),trainable=True)
#     def call(self, inputs):
#         # print(self.W.shape)
#         # print(inputs.shape)
#         # # print("a")
#         x = tf.matmul(inputs, self.W)
#         # print("b")
#         # print(x)
#         # print(x.shape)
#         return x

# class Output_layer(layers.Layer):
#     def __init__(self, units):
#         super(Output_layer, self).__init__()
#         self.units = units

#     def build(self, input_shape):
#         self.d = self.add_weight(shape=(1,), initializer=tf.keras.initializers.GlorotNormal, trainable=False)
#         x = tf.Variable([self.d,-self.d,-self.d,self.d, -self.d,-self.d,self.d,self.d])
#         x = r = tf.reshape(x, shape=(input_shape[-1],self.units))
#         self.W = tf.Variable(x, shape=(input_shape[-1],self.units),trainable=True)
        
#         # self.W = tf.Variable(
#         #     [[self.d,-self.d],
#         #     [-self.d,-self.d], 
#         #      [-self.d,-self.d],
#         #      [self.d,self.d]],shape=(input_shape[-1],self.units),
#         #     trainable=True)

#     def call(self, inputs):
#         # inputs = inputs
#         # W = self.W.transpose()
#         # print(self.W.shape)
#         # print(inputs.shape)
#         # print("a")
#         x = tf.matmul(inputs, self.W)
#         # print("b")
#         # print(x.shape)
#         return x

# class MyReLU(layers.Layer):
#     def __init__(self):
#         super(MyReLU, self).__init__()

#     def call(self, x):
#         return tf.math.maximum(x, 0)

# class MyModel(keras.Model):  # model.fit, model.evalute, model.predict
#     def __init__(self):
#         super(MyModel, self).__init__()
#         self.dense1 = Hidden_layer(4)
#         self.dense2_1 = Output_layer(2)
#         # self.dense2_2 = Output_layer(1)
#         self.relu = MyReLU()

#     def call(self, x):
#         x = self.relu(self.dense1(x))
#         # return {"y1" :self.dense2_1(x),"y2" : self.dense2_2(x)}
#         return self.dense2_1(x)

#     def build_graph(self):
#         x = tf.keras.layers.Input(shape=(1,4))
#         return tf.keras.Model(inputs=[x], outputs=self.call(x))

# model = MyModel()
# model.build_graph().summary()

# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
#               loss='mean_squared_error',
#               metrics = [tf.keras.metrics.MeanSquaredError()]
# )

# model.fit(X_train, y=Y_train, epochs=5, batch_size=5, verbose=2)

# print("output layer weight")
# print(model.layers[1].weights)

# print("hidden layer weight")
# print(model.layers[0].weights)

# predictions = model.predict(X_validation)
# prediction_accuracy(predictions, Y_validation)

In [ ]:
# # Not important Experiment 3 - learned how to give custom kernel constraint
# # how to make constraint work on subclassing api
# # Equivariance NN with subclassing API - experiment with layer constraint
# # These function make use of the subclassing Api and custom layer functionality of tensorflow
# # experiment

# class Hidden_layer(layers.Layer):
#     def __init__(self, units):
#         super(Hidden_layer, self).__init__()
#         self.units = units

#     def build(self, input_shape):
#         self.W = self.add_weight(
#             shape=(input_shape[-1], self.units),initializer=tf.keras.initializers.GlorotNormal, trainable=True)
            
#     def call(self, inputs):
#         x = tf.matmul(inputs, self.W)
#         return x

# class Output_layer_constraint(tf.keras.constraints.Constraint):
#     # def __call__(self, d):
#     #     self.d = d
#     def __call__(self, w):
#         # x = tf.Variable([self.d,-self.d,-self.d,self.d, -self.d,-self.d,self.d,self.d])
#         # w = tf.reshape(x, shape=(4.4))
    
#         return w/w

# class Output_layer(layers.Layer):
#     def __init__(self, units, kernel_constraint):
#         super(Output_layer, self).__init__()
#         self.units = units
#         self.kernel_constraint = kernel_constraint

#     def build(self, input_shape):
#         self.W = self.add_weight(
#             shape=(input_shape[-1], self.units),initializer=tf.keras.initializers.GlorotNormal, trainable=True, constraint=self.kernel_constraint
#         )

#     def call(self, inputs):
#         x = tf.matmul(inputs, self.W)
#         return x

# class MyReLU(layers.Layer):
#     def __init__(self):
#         super(MyReLU, self).__init__()

#     def call(self, x):
#         return tf.math.maximum(x, 0)

# class MyModel(keras.Model):  # model.fit, model.evalute, model.predict
#     def __init__(self):
#         super(MyModel, self).__init__()
#         self.dense1 = Hidden_layer(4)
#         # self.dense2_1 = tf.keras.layers.Dense(2, kernel_constraint=Output_layer_constraint())
#         self.dense2_1 = Output_layer(2, kernel_constraint=Output_layer_constraint())
#         self.relu = MyReLU()

#     def call(self, x):
#         x = self.relu(self.dense1(x))
#         # return {"y1" :self.dense2_1(x),"y2" : self.dense2_2(x)}
#         return self.dense2_1(x)

#     # def build_graph(self):
#     #     x = tf.keras.layers.Input(shape=(1,4))
#     #     return tf.keras.Model(inputs=[x], outputs=self.call(x))

# model = MyModel()
# # model.build_graph().summary()

# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
#               loss='mean_squared_error',
#               metrics = [tf.keras.metrics.MeanSquaredError()]
# )

# model.fit(X_train, y=Y_train, epochs=5, batch_size=5, verbose=2)
# print(model.layers[1].weights)
